In [79]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import janitor 
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm

In [103]:
csmar_monthly = pd.read_csv('/Users/fuhk/Library/CloudStorage/OneDrive-Personal/Data/data/CSMAR/Stock Market/monthly_return/TRD_Mnth.csv')
csmar_monthly.columns = csmar_monthly.columns.str.lower()
csmar_monthly = csmar_monthly[csmar_monthly['markettype'].isin([1,4,16])].reset_index(drop=True)
csmar_monthly['jdate'] = pd.to_datetime(csmar_monthly['trdmnt']) + MonthEnd(0)
csmar_monthly['me'] = csmar_monthly['msmvttl'] * 1000

csmar_firm = pd.read_csv("/Users/fuhk/Library/CloudStorage/OneDrive-Personal/Data/data/CSMAR/firm_information/FS_Combas.csv")
csmar_firm.columns = csmar_firm.columns.str.lower()
csmar_firm = csmar_firm[csmar_firm['typrep'] == "A"]

report_data = pd.read_csv("/Users/fuhk/Library/CloudStorage/OneDrive-Personal/Data/data/CSMAR/firm_information/IAR_Rept.csv")
report_data.columns = report_data.columns.str.lower()
report_data['jdate'] = pd.to_datetime(report_data['accper']) + MonthEnd(0)
report_data['fdate'] = pd.to_datetime(report_data['annodt']) + MonthEnd(0)

1. 时间区间 2002.01.01-2018.12.31
2. 样本使用A股股票（1/4/16）

In [ ]:
#### 不同的版本主要体现在:
## 1. be 的计算方式；
## 2. be和me的匹配方式；

In [104]:
###########
### be ####
###########

csmar_firm['be00'] = csmar_firm['a003000000'] ## equity 

csmar_firm['ps'] = csmar_firm['a003112101'].where(~csmar_firm['a003112101'].isna(), 0)
csmar_firm['mi'] = csmar_firm['a003200000'].where(~csmar_firm['a003200000'].isna(), 0)
csmar_firm['be11'] = csmar_firm['a003000000'] - csmar_firm['mi'] - csmar_firm['ps']
csmar_firm['jdate'] = pd.to_datetime(csmar_firm['accper']) + MonthEnd(0)
csmar_firm99 = csmar_firm[['stkcd','jdate','be00','be11']]

- 方法一, 使用财年进行匹配
-------

In [124]:
dec_be = csmar_firm99[csmar_firm99['jdate'].dt.month == 12]
dec_be['begin_date'] = dec_be['jdate'] + MonthEnd(6)
dec_be['end_date'] = dec_be['begin_date'] + MonthEnd(12)
dec_be = dec_be[['stkcd','begin_date','end_date','be00','be11']]

In [125]:
sample = csmar_monthly.conditional_join(
    dec_be,
    ('stkcd', 'stkcd', '=='),
    ('jdate', 'begin_date', '>='),
    ('jdate', 'end_date', '<'),
    how='left'
    )
sample.drop(columns = {('right','stkcd')}, inplace = True)
sample.columns = sample.columns.droplevel(0)

In [126]:
sample_dec = sample[(sample['jdate']>='2000-01-01') & (sample['jdate']<='2018-12-31')][['stkcd','jdate','be00','be11','me']]

sample_dec['beme_v1'] = sample_dec['be00'] / sample_dec['me']
sample_dec['beme_v1'] = sample_dec['beme_v1'].where(sample_dec['be00']>0,np.nan)

sample_dec['beme_v2'] = sample_dec['be11'] / sample_dec['me']
sample_dec['beme_v2'] = sample_dec['beme_v2'].where(sample_dec['be11']>0,np.nan)

- 方法二：使用季报进行匹配
------

In [132]:
quarter_be = pd.merge(csmar_firm99,report_data[['stkcd','fdate','jdate']],on=['stkcd','jdate'],how="left")
quarter_be.drop(['jdate'],axis=1,inplace=True)

sample = pd.merge(csmar_monthly, quarter_be, left_on=['stkcd','jdate'],right_on=['stkcd','fdate'],how='left')

def fill(group, limit=12):
    return group.ffill(limit=limit)

sample["be00"] = sample.groupby("stkcd")["be00"].transform(lambda x: x.ffill(limit=12))
sample["be11"] = sample.groupby("stkcd")["be11"].transform(lambda x: x.ffill(limit=12))

In [133]:
sample_quarter = sample[(sample['jdate']>='2000-01-01') & (sample['jdate']<='2018-12-31')][['stkcd','jdate','be00','be11','me']]

sample_quarter['beme_v3'] = sample_quarter['be00'] / sample_quarter['me']
sample_quarter['beme_v3'] = sample_quarter['beme_v3'].where(sample_quarter['be00']>0,np.nan)

sample_quarter['beme_v4'] = sample_quarter['be11'] / sample_quarter['me']
sample_quarter['beme_v4'] = sample_quarter['beme_v4'].where(sample_quarter['be11']>0,np.nan)

In [144]:
beme = pd.merge(sample_dec[['stkcd','jdate','beme_v1','beme_v2']],sample_quarter[['stkcd','jdate','beme_v3','beme_v4']],on=['stkcd','jdate'],how='inner').reset_index(drop=True)

- Test
------

In [143]:
correlation_matrix = beme[['beme_v1', 'beme_v2', 'beme_v3', 'beme_v4']].corr()
correlation_matrix

,beme_v1,beme_v2,beme_v3,beme_v4
beme_v1,1.000000,0.989193,0.949980,0.935645
beme_v2,0.989193,1.000000,0.942220,0.947643
beme_v3,0.949980,0.942220,1.000000,0.989497
beme_v4,0.935645,0.947643,0.989497,1.000000


In [181]:
csmar_monthly.sort_values(['stkcd','jdate'],inplace=True)
csmar_monthly['ret_a'] = csmar_monthly.groupby('stkcd')['mretwd'].shift(-1)
csmar_monthly99 = pd.merge(beme,csmar_monthly[['stkcd','jdate','ret_a','me']],on=['jdate','stkcd'],how='left')

csmar_monthly99['lnbeme11'] = np.log(csmar_monthly99['beme_v1'])
csmar_monthly99['lnbeme22'] = np.log(csmar_monthly99['beme_v2'])
csmar_monthly99['lnbeme33'] = np.log(csmar_monthly99['beme_v3'])
csmar_monthly99['lnbeme44'] = np.log(csmar_monthly99['beme_v4'])

csmar_monthly99['lnme'] = np.log(csmar_monthly99['me'])

In [175]:
def FamaMacBeth_summary(dataset,
                        reg_list,
                        var_order,
                        NW_lags,
                        column_names=None,
                        params_format='{:.4f}',
                        tvalues_format='{:.2f}',
                        add_significance=False):  
    """
    Fama-MacBeth_summary():提供每个回归系数的均值、t值、显著性标记、R² 和观测数

    参数：
    ----------
    dataset : pd.DataFrame
        包含回归所需数据的 DataFrame
        例如
                               mret    beta
        date       stkcd                    
        2000-01-31  1          0.061891  1.502101
                    2          0.161126  1.432630
                    5          0.365894  1.002413

        2000-02-28  1          0.072456  1.592101
                    2          0.175646  1.472630
                    5          0.395894  1.112413
    
    reg_list : list of lists
        一个包含多个回归模型的列表
        注意,第一个元素是 y 变量, 并且不需要常数项
        例如：[['ret', 'factor1', 'factor2'], ['ret', 'factor3', 'factor4']]

    var_order : list
        定义了输出表格每一行变量的输出顺序
        注意,务必包含所有的 X 变量

    NW_lags : int
        NW 滞后期数

    column_names : list, optional
        回归模型名称的列表，默认是 `[(1), (2), ..., (n)]`

    params_format : str, optional
        系数的显示格式,默认是 '{:.4f}'，表示四位小数。可以自定义小数位数。

    tvalues_format : str, optional
        t值的显示格式,默认是 '{:.2f}'，表示两位小数。可以自定义小数位数。

    add_significance : bool, optional
        是否添加显著性标记（'***'、'**'、' *'），默认是 `False`

    返回：
    -------
    pd.DataFrame
        包含回归系数、t值、显著性标记、R² 和观测数的 DataFrame。 
        行为每个回归因子和 t 值的系数，列为每个回归模型的名称。
    注意：
    1. 这里的R2是每一个横截面回归的平均R2

    --------
    # 示例
    result = FamaMacBeth_summary(dataset=df,
                                 reg_list=[['const', 'factor1', 'factor2'], ['const', 'factor3', 'factor4']],
                                 var_order=['factor1', 'factor2', 'factor3', 'factor4'],
                                 NW_lags=5,
                                 add_significance=True)

    """
    # 生成表格框架
    rows = sum([[var, f'{var}_t'] for var in ['const'] + var_order], []) + ['r2', 'obs']
    if column_names is None:
        column_names = [f'({i+1})' for i in range(len(reg_list))]
    show = pd.DataFrame(index=rows, columns=column_names)

    # 遍历每个回归模型
    for reg, reg_name in zip(reg_list, column_names):
        _subset = dataset.loc[:, reg].copy().dropna()

    # 横截面回归
        param_list = []
        for date, group in _subset.groupby('date'):
            result = sm.OLS(group[reg[0]], sm.add_constant(group[reg[1:]])).fit()
            params = result.params
            param_list.append(params)
            params['r2'] = result.rsquared
            params['obs'] = result.nobs
        time_series = pd.DataFrame(param_list)
        show.loc['r2', reg_name] = time_series['r2'].mean()
        show.loc['obs', reg_name] = time_series['obs'].sum()
        
        _subseries = time_series.iloc[:, :-2]
        for col in _subseries.columns:
            result = sm.OLS(time_series[col], np.ones(len(time_series[col]))).fit(cov_type="HAC", cov_kwds={"maxlags": NW_lags})
            coef = result.params[0]
            t_value = result.tvalues[0]

            if add_significance:
                if abs(t_value) > 2.576:
                    coef_str = f'{params_format.format(coef)}***'
                elif abs(t_value) > 1.96:
                    coef_str = f'{params_format.format(coef)}**'
                elif abs(t_value) > 1.645:
                    coef_str = f'{params_format.format(coef)}*'
                else:
                    coef_str = f'{params_format.format(coef)}'
            else:
                coef_str = f'{params_format.format(coef)}'
            t_value_str = f'({tvalues_format.format(t_value)})'


            show.loc[f'{col}', reg_name] = coef_str
            show.loc[f'{col}_t', reg_name] = t_value_str
    
    show.index = ["" if '_t' in idx else idx for idx in show.index]
    show = show.fillna('')

    return show

In [182]:
reg_list=[['ret_a','lnbeme11'],
          ['ret_a','lnbeme22'],
          ['ret_a','lnbeme33'],
          ['ret_a','lnbeme44'],    
          ['ret_a','lnbeme11','lnme'],
          ['ret_a','lnbeme22','lnme'],
          ['ret_a','lnbeme33','lnme'],
          ['ret_a','lnbeme44','lnme'],  
]
var_order = ['lnme','lnbeme11','lnbeme22','lnbeme33','lnbeme44']
NW_lags = 5

In [183]:
csmar_monthly99.rename(columns={"jdate": "date"}, inplace=True)
csmar_monthly99.set_index(['date','stkcd'],inplace=True)
FamaMacBeth_summary(csmar_monthly99,
                    reg_list,
                    var_order,
                    NW_lags)

,(1),(2),(3),(4),(5),(6),(7),(8)
const,0.0191,0.0192,0.0187,0.0189,0.1516,0.1469,0.1486,0.1484
,(2.33),(2.34),(2.33),(2.34),(3.43),(3.40),(3.53),(3.51)
lnme,,,,,-0.0060,-0.0058,-0.0058,-0.0058
,,,,,(-3.30),(-3.25),(-3.36),(-3.34)
lnbeme11,0.0038,,,,0.0034,,,
,(3.45),,,,(3.14),,,
lnbeme22,,0.0038,,,,0.0034,,
,,(3.55),,,,(3.35),,
lnbeme33,,,0.0041,,,,0.0043,
,,,(3.33),,,,(3.89),
